In [13]:
import json
import numpy as np
from collections import defaultdict
from dataclasses import dataclass, field
import os
import pickle
import re

from get_data import TbcaUnit

from get_data import *

# --- 1. Nova Lógica de Parsing (De trás para frente) ---
def carregar_mapa_codigos_limpo(caminho_arquivo: str) -> dict:
    """
    Lê o arquivo mapa_codigo.txt varrendo de trás para frente.
    Enquanto o 'token' for um código (inicia com BR ou é -), ele é removido.
    O que sobrar é o nome do alimento.
    """
    
    if os.path.exists("mapa_codigo.pkl"):
        with open("mapa_codigo.pkl", "rb") as f:
            return pickle.load(f)
    
    mapa = {}
    
    if not os.path.exists(caminho_arquivo):
        print(f"Erro: Arquivo {caminho_arquivo} não encontrado.")
        return mapa

    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            for linha in f:
                linha = linha.strip()
                if not linha: 
                    continue
    
                
                # 2. Divide a linha em palavras (tokens)
                partes = linha.split()
                
                codigos_encontrados = []
                
                while partes:
                    ultimo_token = partes[-1]
                    
            
                    eh_codigo = (ultimo_token.startswith("BR") or 
                                 ultimo_token == "-" or 
                                 ("," in ultimo_token and "BR" in ultimo_token))
                    
                    if eh_codigo:
                        token = partes.pop()
                        if token != "-":
                            sub_cods = [c.strip() for c in token.split(',')]
                            for c in sub_cods:
                                if c and c.startswith("BR"):
                                    codigos_encontrados.append(c)
                    else:
                        break
                
                
                if partes:
                    nome_alimento = " ".join(partes)
                    mapa[nome_alimento] = codigos_encontrados
                        
    except Exception as e:
        print(f"Erro ao processar o arquivo mapa_codigo: {e}")

    with open("mapa_codigo.pkl", "wb") as f:
        pickle.dump(mapa, f)
    
    return mapa

# --- 2. Função Auxiliar de Cálculo de Nutrientes de um Item ---
def calcular_nutrientes_item(nome_alimento: str, quantidade_g: float, 
                             mapa_codigos: dict, dados_tbca: dict) -> dict:
    """
    Calcula os nutrientes considerando média de múltiplos códigos.
    """
    codigos = mapa_codigos.get(nome_alimento)
    
    if not codigos:
        # Se não tem código (lista vazia ou None), retorna vazio
        return {}

    nutrientes_acumulados = defaultdict(float)
    codigos_validos_encontrados = 0

    for code in codigos:
        # Tenta pegar o objeto TbcaUnit do dicionário carregado do pickle
        unidade = dados_tbca.get(code)
        
        # Só processa se o código existir no banco TBCA e tiver nutrientes extraídos
        if unidade and unidade.nutrientes:
            codigos_validos_encontrados += 1
            for nutriente, valor_por_100g in unidade.nutrientes.items():
                # Regra de 3: (ValorTBCA * QuantidadeIngerida) / 100
                qtd_real = (valor_por_100g * quantidade_g) / 100.0
                nutrientes_acumulados[nutriente] += qtd_real
    
    if codigos_validos_encontrados == 0:
        return {}

    # Média Aritmética: Divide a soma total pelo número de códigos usados
    nutrientes_finais = {k: v / codigos_validos_encontrados for k, v in nutrientes_acumulados.items()}
    
    return nutrientes_finais

# --- 3. Função Principal de Estatísticas ---
def calcular_estatisticas_dietas(caminho_pickle_tbca: str = "mapa_tbca_completo.pkl", 
                                 caminho_gbt_json: str = "rcsc/gbt.json", 
                                 caminho_mapa_txt: str = "rcsc/mapa_codigo.txt"):
    
    # 1. Carrega o "Banco de Dados" TBCA
    if not os.path.exists(caminho_pickle_tbca):
        print("Erro: Arquivo pickle da TBCA não encontrado. Rode o scraping primeiro.")
        return
        
    print("Carregando dados da TBCA...")
    with open(caminho_pickle_tbca, "rb") as f:
        dados_tbca = pickle.load(f)

    # 2. Carrega o Mapa de Códigos com a nova lógica
    print("Processando mapa de códigos...")
    mapa_codigos = carregar_mapa_codigos_limpo(caminho_mapa_txt)
    
    # 3. Carrega as Dietas
    if not os.path.exists(caminho_gbt_json):
        print(f"Erro: Arquivo JSON {caminho_gbt_json} não encontrado.")
        return

    with open(caminho_gbt_json, 'r', encoding='utf-8') as f:
        dietas = json.load(f)

    print("Calculando estatísticas...")
    
    # Estruturas para armazenar: dict[Categoria][Nutriente] = [lista de valores]
    totais_diarios = defaultdict(lambda: defaultdict(list))
    totais_refeicao = defaultdict(lambda: defaultdict(list))
    
    # Loop pelas dietas (Planos)
    for plano in dietas:
        # Loop pelos Dias ("1", "2", ...)
        for dia, refeicoes in plano.items():
            nutrientes_do_dia = defaultdict(float)
            
            # Loop pelas Refeições ("Café", "Almoço"...)
            for nome_refeicao, itens in refeicoes.items():
                nutrientes_da_refeicao = defaultdict(float)
                
                for item in itens:
                    nome = item.get('alimento')
                    try:
                        qtd = float(item.get('quantidade', 0))
                    except:
                        qtd = 0
                    
                    # O coração do cálculo
                    nutris_item = calcular_nutrientes_item(nome, qtd, mapa_codigos, dados_tbca)
                    
                    # Acumula
                    for n, v in nutris_item.items():
                        nutrientes_da_refeicao[n] += v
                        nutrientes_do_dia[n] += v
                
                # Salva o total desta refeição específica na lista de histórico
                for n, v in nutrientes_da_refeicao.items():
                    totais_refeicao[nome_refeicao][n].append(v)
            
            # Salva o total deste dia na lista de histórico
            for n, v in nutrientes_do_dia.items():
                totais_diarios["Geral"][n].append(v)

    # --- Exibição dos Resultados (Média ± Desvio Padrão) ---
    
    def formatar_saida(titulo, dados_dict):
        print(f"\n{'='*40}")
        print(f"ESTATÍSTICAS: {titulo}")
        print(f"{'='*40}")
        
        nutrientes_foco = ["Energia", "Carboidrato", "Proteína", "Lipídeos", "Fibra alimentar", "Sódio"]
        
        for categoria, nutris_map in dados_dict.items():
            print(f"\n>> {categoria.upper()}")
            
            todos_keys = set(nutris_map.keys())
            # Ordena: Prioridades primeiro, depois alfabético
            chaves_ordenadas = [k for k in nutrientes_foco if k in todos_keys]
            chaves_ordenadas += sorted(list(todos_keys - set(nutrientes_foco)))
            
            count_exibidos = 0
            for nutri in chaves_ordenadas:
                valores = nutris_map[nutri]
                if not valores: continue
                
                media = np.mean(valores)
                desvio = np.std(valores)
                
                # Filtro simples para não exibir micronutrientes zerados ou irrelevantes
                if media > 0.1:
                    unidade = "kcal" if nutri == "Energia" else "g" # Simplificação visual
                    if nutri == "Sódio": unidade = "mg"
                    
                    print(f"  {nutri:.<20} {media:>8.2f} ± {desvio:<6.2f} {unidade}")
                    count_exibidos += 1
            
            if count_exibidos == 0:
                print("  (Sem dados significativos calculados)")

    formatar_saida("DIÁRIO (Média por dia)", totais_diarios)
    formatar_saida("POR REFEIÇÃO (Média por refeição)", totais_refeicao)


carregar_mapa_codigos_limpo("mapa_codigo.txt")

# calcular_estatisticas_dietas(
#     caminho_pickle_tbca="mapa_tbca_completo.pkl", # Arquivo gerado pelo scraping
#     caminho_gbt_json="rcsc/gbt.json",
#     caminho_mapa_txt="rcsc/mapa_codigo.txt"
# )

{'Leite Integral': ['BRC0043G'],
 'Aveia em Flocos': ['BRC0021A'],
 'Maçã': ['BRC0023C'],
 'Banana Prata': ['BRC0011C'],
 'Amendoim torrado sem sal': ['BRC0021T'],
 'Arroz Cozido': ['BRC0209A'],
 'Feijão Cozido': ['BRC0091T'],
 'Peito de Frango Grelhado': ['BRC0230F'],
 'Salada de Folhas e Tomate': ['BRC0396B'],
 'Azeite para tempero': [],
 'Pão Francês': ['BRC0002A'],
 'Queijo Minas Padrão': ['BRC0056G'],
 'Batata Doce Cozida': ['BRC0042B'],
 'Ovos Cozidos': ['BRC0010J'],
 'Salada de Brócolis e Cenoura Cozida': ['BRC0643B', 'BRC0504B'],
 'Salada de Brócolis': ['BRC0643B'],
 'Salada de Cenoura': ['BRC0504B'],
 'Chá de Ervas': [],
 'Ovos Mexidos': ['BRC0028J'],
 'Azeite para preparo dos ovos': [],
 'Iogurte Natural Integral': ['BRC0011G'],
 'Mel': ['BRC0028K'],
 'Carne Moída Magra (patinho)': ['BRC0441F'],
 'Salada Mista (alface, tomate, pepino)': ['BRC0396B'],
 'Leite em Pó Integral': ['BRC0041G'],
 'Mandioca Cozida (Aipim)': ['BRC0908B'],
 'Filé de Frango Grelhado': ['BRC0230F'],
 'Ab